<a href="https://colab.research.google.com/github/fmbento/Jupyter-Notebooks_ad-hoc-solutions/blob/main/ESTGA_extrai_dados_da_Bib_Recomendada_por_UC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!touch URLs_ESTGA_Lic_UCs.txt

In [2]:
%%capture
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


Fazer paste dos links das UCs no ficheiro criado acima, enquanto a célula acima corre -- não é preciso esperar que ela termine.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

import csv
import time
from google.colab import files

import re

# Setup chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Function to extract information from a single page
def extract_info(url):
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    docente = ""
    codigo = ""

    # wait = WebDriverWait(driver, 10)  # wait for up to 10 seconds
    time.sleep(5)  # wait for 10 seconds -- safe value, despite 2 seconds did work initially

    UC = driver.find_element(By.XPATH, "//h1").text


    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find the <H3> tag with the desired text and get the next <a>
    h3_tag_doc = soup.find('h3', string='Docente responsável:')
    try:
        docente = h3_tag_doc.find_next_sibling('a').text
    except Exception as e:
        print(f"Sem Docente: {e} - " + url)


    # Find the <H3> tag with the desired text and get the next <div>
    h3_tag_cod = soup.find('h3', string='Código:')
    codigo = h3_tag_cod.find_next_sibling('div').text


    print(docente)
    print(codigo)


    # Find the <H2> tag with the desired text and get the next <div>
    h2_tag = soup.find('h2', string='Bibliografia recomendada')

    bibrec = ""
    try:
        next_div = h2_tag.find_next_sibling('div')
    except Exception as e:
        print(f"Sem bib Rec: {e} - " + url)

    # Extract and print the text within the <div>
    try:
        bibrec = next_div.text
    except Exception as e:
        print()
    print(bibrec)
    print("----------------------------------------------------")

    driver.quit()

    return UC,docente,codigo,bibrec,url

# Read URLs from file and extract information
with open('URLs_ESTGA_Lic_UCs.txt', 'r') as url_file, open('ESTGA_UCs_BibRec.csv', 'w') as output_file:
    csv_writer = csv.writer(output_file)
    csv_writer.writerow(["UC","docente","codigo","BibRec","URL"])  # Write header

    for url in url_file:
        UC, docente, codigo, bibrec, url = extract_info(url.strip())
        csv_writer.writerow([UC,docente,codigo,bibrec,url])


# AutoDonwloads 'SAUNE_DirsCursos.csv'
files.download('ESTGA_UCs_BibRec.csv')

## E é isto... feito!

Abaixo é código solto para a tentativa de fazer o parsing das referências bibliográficas -- não funciona.

In [ ]:
       # Expressão regular para extrair os metadados da referencias bib em bibrec
        regex = r"^(.*), \((\d{4})\)\. (.*). (\d+ª Edição), (.*)\.$"

        # Extrai os metadados
        referencias = bibrec.splitlines()
        for ref in referencias:
            # print(ref)
            match = re.match(r'^(.*), \((\d{4})\)\. (.*), (.*)$', ref)
            if match:
                groups = match.groups()
                authors = groups[0]
                year = groups[1]
                title = groups[2]
                rest = groups[3]

                edition_publisher = rest.rsplit(',', 1)
                if len(edition_publisher) == 2:
                    edition, publisher = edition_publisher
                else:
                    edition = edition_publisher[0]
                    publisher = ''

                print(f"Autores: {authors}")
                print(f"Ano: {year}")
                print(f"Título: {title}")
                print(f"Edição: {edition.strip()}")
                print(f"Editora: {publisher.strip()}")
                print()